# Quadruped Timing (Soft + I)

Builds and runs the quadruped benchmark with command-line options, then plots timing for the selected methods.

In [1]:
from pathlib import Path
import re
import subprocess

import pandas as pd
import plotly.graph_objects as go
import gtdynamics as gtd


def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "CMakeLists.txt").exists() and (candidate / "examples" / "example_constraint_manifold").exists():
            return candidate
    raise RuntimeError("Could not find GTDynamics repo root.")


repo_root = find_repo_root(Path.cwd().resolve())
build_dir = repo_root / "build"
example_bin = build_dir / "examples" / "example_constraint_manifold" / "example_constraint_manifold_quadruped_mp"
data_dir = Path(gtd.DATA_PATH)

num_steps = 15
methods = "soft,i"

repo_root, build_dir, data_dir, num_steps, methods


(PosixPath('/Users/dellaert/git/GTDynamics'),
 PosixPath('/Users/dellaert/git/GTDynamics/build'),
 PosixPath('/Users/dellaert/git/GTDynamics/data'),
 15,
 'soft,i')

In [2]:
subprocess.run(["make", "-j6", "example_constraint_manifold_quadruped_mp"], cwd=build_dir, check=True)
run_cmd = [str(example_bin), "--num-steps", str(num_steps), "--methods", methods]
run = subprocess.run(run_cmd, cwd=build_dir, check=True, text=True, capture_output=True)
print(run.stdout)
run_stdout = run.stdout


[ 95%] Built target gtdynamics
[100%] Built target example_constraint_manifold_quadruped_mp
Using num_steps=15
Methods: soft=1, f=0, i=1
collocation costs:	134.298
boundary costs:	1.62008e-24
min torque costs:	60.7941
friction cone costs:	0
soft constraints:
[BENCH] Soft Constraint: f_dim=7366, v_dim=7200, time_s=0.099, iters=7, constraint_l2=3.729e-11, cost=195.092
collocation costs:	134.298
boundary costs:	1.62008e-24
min torque costs:	60.7941
friction cone costs:	0
constraint manifold basis variables feasible: skipped
constraint manifold basis variables infeasible:
[BENCH] \textbf{Constraint Manifold (I)}: f_dim=454, v_dim=288, time_s=1.054, iters=0, constraint_l2=1.430e-09, cost=3517.25
collocation costs:	3488.49
boundary costs:	0.334668
min torque costs:	28.4309
friction cone costs:	0
& Soft Constraint & $7366 \times 7200$ & 0.099 & 7 & 3.73e-11 & 195.09\\
& \textbf{Constraint Manifold (I)} & $454 \times 288$ & 1.054 & 0 & 1.43e-09 & 3517.25\\



In [3]:
pattern = re.compile(r"\[BENCH\]\s*(?:\\textbf\{)?([^:}]+)(?:\})?:.*time_s=([0-9.]+)")
timings = {}
for line in run_stdout.splitlines():
    m = pattern.search(line)
    if not m:
        continue
    name = m.group(1).strip()
    time_s = float(m.group(2))
    if name == "Soft Constraint":
        timings["Soft"] = time_s
    elif "Constraint Manifold (I)" in name:
        timings["I"] = time_s

timing_df = pd.DataFrame({"method": list(timings.keys()), "time_s": list(timings.values())})
if timing_df.empty:
    raise RuntimeError("No timing rows found in benchmark output.")

order = ["Soft", "I"]
timing_df["method"] = pd.Categorical(timing_df["method"], categories=order, ordered=True)
timing_df = timing_df.sort_values("method")

fig = go.Figure(go.Bar(
    x=timing_df["method"],
    y=timing_df["time_s"],
    text=timing_df["time_s"].map(lambda v: f"{v:.3f}s"),
    textposition="outside",
))
fig.update_layout(
    title=f"Quadruped Timing ({methods}, num_steps={num_steps})",
    xaxis_title="Method",
    yaxis_title="Time [s]",
    width=700,
    height=450,
)
fig.show()
timing_df


,method,time_s
0,Soft,0.099
1,I,1.054


In [4]:
from plotly.subplots import make_subplots


def load_traj_arrays(file_path: Path):
    df = pd.read_csv(file_path)
    jangles = df[[str(j) for j in range(12)]].to_numpy()
    jvels = df[[f"{j}.1" for j in range(12)]].to_numpy()
    jaccels = df[[f"{j}.2" for j in range(12)]].to_numpy()
    jtorques = df[[f"{j}.3" for j in range(12)]].to_numpy()
    return jangles, jvels, jaccels, jtorques


cm_file = "cm_traj.csv" if (data_dir / "cm_traj.csv").exists() else "cm_infeas_traj.csv"
cm_label = "manifold" if cm_file == "cm_traj.csv" else "manifold (I)"
traj_specs = [("init", "init_traj.csv"), (cm_label, cm_file), ("soft", "soft_traj.csv")]
method_colors = {
    "init": "#1f77b4",
    "manifold": "#d62728",
    "manifold (I)": "#d62728",
    "soft": "#2ca02c",
}


def plot_joint_comparison(idx: int, joint_name: str):
    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=("q", "v", "a", "T"),
        shared_xaxes=True,
        vertical_spacing=0.14,
    )

    for label, filename in traj_specs:
        jangles, jvels, jaccels, jtorques = load_traj_arrays(data_dir / filename)
        series = [jangles[:, idx], jvels[:, idx], jaccels[:, idx], jtorques[:, idx]]
        color = method_colors.get(label)

        for k, y in enumerate(series):
            row = 1 + k // 2
            col = 1 + k % 2
            fig.add_trace(
                go.Scatter(
                    y=y,
                    mode="lines",
                    name=label,
                    legendgroup=label,
                    showlegend=(k == 0),
                    line={"color": color},
                ),
                row=row,
                col=col,
            )

    fig.update_xaxes(title_text="step", row=2, col=1)
    fig.update_xaxes(title_text="step", row=2, col=2)
    fig.update_layout(
        title=f"{joint_name} joint trajectories",
        width=950,
        height=620,
        legend_title="trajectory",
    )
    fig.show()


plot_joint_comparison(0, "hip")
plot_joint_comparison(1, "knee")
print("Used trajectory files:", [f for _, f in traj_specs])


Used trajectory files: ['init_traj.csv', 'cm_infeas_traj.csv', 'soft_traj.csv']
